$$
Z = \frac{D}{\sqrt{\frac{s^{2}_{D}}{N}}}
$$

$$
Z = \frac{\bar{x_{1}}+\bar{x_{2}}}{\sqrt{\frac{s^{2}_{1}}{N_{1}} + \frac{s^{2}_{2}}{N_{2}}}}
$$

$$
\chi^{2} = \sum_{i}\frac{(y_{i} - y(h_{i}))^{2}}{\sigma ^{2}}
$$

$$
r = \frac{\vec{A}\vec{B}}{|A||B|}
$$

In [2]:
#Import libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

#Open and read from file
file_name = "PH 336_SDL_ROM_2024-03-13.csv"   #update this
data = pd.read_csv(file_name,encoding='unicode_escape')

#Romney Data (see if we cant abstract a bit more efficiently)
Rom_base_list = np.array(data["CountRate(CPM)"][6:12].astype(float))  
Rom_floor1_list = np.array(data["CountRate(CPM)"][15:23].astype(float))
Rom_floor2_list = np.array(data["CountRate(CPM)"][26:41].astype(float))

#Def Zscore test
def ZScore(x_array_1,x_array_2):
    #Calculate the number of values in both arrays
    N1 = len(x_array_1)
    N2 = len(x_array_2)

    #Calculate the mean of both arrays
    x1_mean = sum(x_array_1)/N1
    x2_mean = sum(x_array_2)/N2

    #Calculate the stdev of both
    stdev_x1 = np.sqrt((sum(x_array_1-x1_mean)**2)/(N1-1))
    stdev_x2 = np.sqrt((sum(x_array_2-x2_mean)**2)/(N2-1))

    return (x1_mean + x2_mean)/(np.sqrt((stdev_x1*stdev_x1/N1) + (stdev_x2 * stdev_x2/N2)))





def main():
    print("Z_score = ",ZScore(Rom_base_list,Rom_floor1_list))
    print(Rom_base_list)
    print(Rom_floor1_list)
    print(Rom_floor2_list)
    return

if __name__ == "__main__":
    main()

Z_score =  inf
[452. 456. 342. 354. 405. 352.]
[598. 599. 648. 742. 739. 761. 774. 764.]
[775. 726. 537. 468. 561. 600. 653. 639. 682. 789. 839. 776. 762. 894.
 841.]


C:\Users\slick\AppData\Local\Temp\ipykernel_371688\1740242096.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  return (x1_mean + x2_mean)/(np.sqrt((stdev_x1*stdev_x1/N1) + (stdev_x2 * stdev_x2/N2)))
